# Exemplo de Uso do Agent com LangGraph

Este notebook demonstra como criar e utilizar um agente de pesquisa utilizando o LangGraph. 
O agente será configurado para responder perguntas sobre o clima em uma determinada cidade, usando um modelo de linguagem GPT-4 e uma ferramenta de busca na web.


In [ ]:

# Instalação das bibliotecas necessárias (se ainda não instaladas)
!pip install -qU langchain langchain_openai langgraph langchain_community graphviz python-dotenv


## Importações e Configuração Inicial

In [ ]:

from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage

# Inicializa o modelo de linguagem GPT-4
model = ChatOpenAI(model="gpt-4")

# Configura a ferramenta de busca na web
search_tool = TavilySearchResults(max_results=2)

# Configura o sistema de persistência
memory = SqliteSaver.from_conn_string(":memory:")


## O que o `Agent` faz?
O `Agent` é responsável por processar entradas, tomar decisões baseadas em um grafo de estados, e interagir com ferramentas para fornecer respostas ou executar ações.
Aqui, o `Agent` usará um modelo GPT-4 para interpretar uma pergunta e buscar na web informações sobre o clima.


In [ ]:

from typing import TypedDict, Annotated
import operator

# Define a estrutura do estado do agente
class AgentState(TypedDict):
    messages: Annotated[list[HumanMessage], operator.add]

class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        self.model = model
        self.tools = tools
        self.checkpointer = checkpointer

        # Configura o grafo de estados
        self.graph = StateGraph(AgentState)
        self.graph.add_node("llm", self.call_openai)
        self.graph.add_node("action", self.take_action)
        self.graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})

    def call_openai(self, state):
        messages = state["messages"]
        response = self.model(messages)
        messages.append(response)
        state["messages"] = messages
        return state

    def exists_action(self, state):
        # Verifica se há necessidade de realizar uma ação baseada na resposta do modelo
        last_message = state["messages"][-1]
        if "search" in last_message.content.lower():
            return True
        return False

    def take_action(self, state):
        # Executa a ação de busca e atualiza o estado
        search_results = self.tools["search"].run(query="current weather in SF")
        state["messages"].append(ToolMessage(content=search_results))
        return state

    def run(self, prompt):
        messages = [HumanMessage(content=prompt)]
        thread = {"configurable": {"thread_id": "1"}}
        for event in self.graph.stream({"messages": messages}, thread):
            for v in event.values():
                print(v.content)


## Como Usar o `Agent`
Neste exemplo, vamos perguntar sobre o clima em San Francisco e deixar o `Agent` buscar e responder a pergunta usando o modelo GPT-4 e a ferramenta de busca na web.


In [ ]:

# Inicializando o agente com as ferramentas e o modelo configurados
tools = {"search": search_tool}
abot = Agent(model=model, tools=tools, checkpointer=memory)

# Prompt para perguntar sobre o clima em São
prompt = "What is the current weather in San Paulo?"

# Executando o agente com o prompt
abot.run(prompt)
